# CIFAR-10 Colab Notebook
This notebook demonstrates how to download, process, and use the CIFAR-10 dataset.

In [ ]:
# Installing the Kaggle API to download datasets
!pip install kaggle

In [ ]:
# Setting up the Kaggle API key
!mkdir -p ~/.kaggle  # Create a directory for the API key
!cp kaggle.json ~/.kaggle/  # Copy the Kaggle API key to the directory
!chmod 600 ~/.kaggle/kaggle.json  # Secure the API key file

In [ ]:
# Downloading the CIFAR-10 dataset from Kaggle
!kaggle competitions download -c cifar-10

In [ ]:
# Verifying the downloaded files
!ls

In [ ]:
# Extracting the CIFAR-10 dataset
from zipfile import ZipFile
dataset = '/content/cifar-10.zip'  # Path to the zip file

with ZipFile(dataset, 'r') as zip:
    zip.extractall()  # Extract all contents of the zip file
    print('The dataset is extracted')

In [ ]:
# Checking the extracted contents
!ls

In [ ]:
# Installing py7zr for handling 7z compressed files
!pip install py7zr

In [ ]:
# Extracting the train.7z file
import py7zr

archive = py7zr.SevenZipFile('/content/train.7z', mode='r')  # Opening the archive
archive.extractall()  # Extracting all contents
archive.close()  # Closing the archive

In [ ]:
# Verifying extracted files again
!ls

In [ ]:
# Importing necessary libraries
import os
import numpy as np
import pandas as pd
from PIL import Image  # For image processing
import matplotlib.pyplot as plt  # For plotting
import matplotlib.image as mpimg  # For loading images
from sklearn.model_selection import train_test_split  # For splitting data

In [ ]:
# Listing all image filenames in the train folder
filenames = os.listdir('/content/train')
print(len(filenames))  # Printing the number of files
print(filenames[0:5])  # Displaying a sample of filenames

In [ ]:
# Reading labels from the CSV file
labels_df = pd.read_csv('/content/trainLabels.csv')

# Inspecting the labels dataset
print(labels_df.shape)  # Shape of the labels dataset
print(labels_df.head())  # First 5 rows of the dataset

In [ ]:
# Mapping string labels to numerical values
labels_dictionary = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3,
                     'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
labels = [labels_dictionary[i] for i in labels_df['label']]

In [ ]:
# Visualizing a sample image
import cv2
from google.colab.patches import cv2_imshow  # For displaying images in Colab

img = cv2.imread('/content/train/7796.png')  # Loading an image
cv2_imshow(img)  # Displaying the image

In [ ]:
# Converting images to NumPy arrays for processing
train_data_folder = '/content/train/'
data = []

for id in labels_df['id']:
    image = Image.open(train_data_folder + str(id) + '.png')  # Load image
    image = np.array(image)  # Convert to NumPy array
    data.append(image)  # Append to the list

In [ ]:
# Converting the list of images and labels to NumPy arrays
X = np.array(data)  # Features
Y = np.array(labels)  # Labels

In [ ]:
# Splitting data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
# Scaling the data by normalizing pixel values to [0, 1]
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

In [ ]:
# Creating a basic neural network model
import tensorflow as tf
from tensorflow import keras

num_of_classes = 10

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 32, 3)),  # Flatten the input images
    keras.layers.Dense(64, activation='relu'),  # Dense layer with ReLU activation
    keras.layers.Dense(num_of_classes, activation='softmax')  # Output layer with Softmax
])

# Compiling the neural network
model.compile(optimizer='adam',  # Optimizer
              loss='sparse_categorical_crossentropy',  # Loss function
              metrics=['acc'])  # Evaluation metric

In [ ]:
# Training the neural network
model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=10)

In [ ]:
# Implementing transfer learning using ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import models, layers, optimizers

convolutional_base = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Building a new model on top of ResNet50
model = models.Sequential([
    layers.UpSampling2D((2, 2)),
    layers.UpSampling2D((2, 2)),
    layers.UpSampling2D((2, 2)),  # Upsample to match ResNet input
    convolutional_base,  # Pre-trained ResNet
    layers.Flatten(),  # Flatten features
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(num_of_classes, activation='softmax')  # Output layer
])

# Compiling the ResNet-based model
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# Training the ResNet-based model
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=10)

In [ ]:
# Evaluating the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

In [ ]:
# Plotting training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation accuracy
plt.plot(history.history['acc'], label='train accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()